In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim

import matplotlib.pyplot as plt

!pip install -q torchviz
from torchviz import make_dot

import random
import scipy

In [ ]:
for df in [train, test]:
    # Extract the 10 letters of f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    # unique_characters feature is from https://www.kaggle.com/code/cabaxiom/tps-may-22-eda-lgbm-model
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
    # Feature interactions: create three ternary features
    # Every ternary feature can have the values -1, 0 and +1
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

y = train.target.values
train = train.drop(['f_27', 'target'], axis = 1)
test = test.drop('f_27', axis = 1)
test.head()

In [ ]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [ ]:
# Definition of a DNN Model class
class DNN(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(DNN, self).__init__()
        
        
        # Fully Connected Layer
        self.layers = nn.Sequential(nn.Linear(input_size, 64),
                                nn.SiLU(),
                                nn.Linear(64, 64),
                                nn.SiLU(),
                                nn.Linear(64, 64),
                                nn.SiLU(),
                                nn.Linear(64, 64),
                                nn.SiLU(),
                                nn.Linear(64, 16),
                                nn.SiLU(),
                                nn.Linear(16, output_size),
                                nn.Sigmoid()
                               )
        
    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = self.layers(x)
        return x

In [ ]:
model_dnn_plot = DNN(train.shape[1])
x_plot = torch.randn(1, train.shape[1])
y_plot = model_dnn_plot(x_plot)
make_dot(y_plot.mean(), params=dict(model_dnn_plot.named_parameters()))

In [ ]:
# Validation function
def validation(model, loader, criterion, device="cpu"):
    model.eval()
    loss = 0
    preds_all = torch.LongTensor()
    labels_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x, labels in loader:
            labels_all = torch.cat((labels_all, labels), dim=0)
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            output = model.forward(batch_x)
            loss += criterion(output,labels).item()
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
    total_loss = loss/len(loader)
    auc_score = roc_auc_score(labels_all, preds_all)
    return total_loss, auc_score

In [ ]:
# Training function
def train_model(model, trainloader, validloader, criterion, optimizer, 
                scheduler, epochs=20, device="cpu", print_every=1):
    model.to(device)
    best_auc = 0
    best_epoch = 0
    
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the learning rates in each eporch
    learning_rates = []
    
    for e in range(epochs):
        model.train()
        
        for batch_x, labels in trainloader:
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            # Training 
            optimizer.zero_grad()
            output = model.forward(batch_x)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
        # at the end of each epoch calculate loss and auc score:
        model.eval()
        train_loss, train_auc = validation(model, trainloader, criterion, device)
        valid_loss, valid_auc = validation(model, validloader, criterion, device)
        
        #### record loss and learning rate
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        learning_rates.append(scheduler._last_lr)
        
        if valid_auc > best_auc:
            best_auc = valid_auc
            best_epoch = e
            torch.save(model.state_dict(), "best-state.pt")
        if e % print_every == 0:
            to_print = "Epoch: "+str(e+1)+" of "+str(epochs)
            to_print += ".. Train Loss: {:.4f}".format(train_loss)
            to_print += ".. Valid Loss: {:.4f}".format(valid_loss)
            to_print += ".. Valid AUC: {:.3f}".format(valid_auc)
            print(to_print)
    # After Training:
    model.load_state_dict(torch.load("best-state.pt"))
    to_print = "\nTraining completed. Best state dict is loaded.\n"
    to_print += "Best Valid AUC is: {:.4f} after {} epochs".format(best_auc,best_epoch+1)
    print(to_print)
    return train_losses, valid_losses, learning_rates

In [ ]:
# Prediction function
def prediction(model, loader, device="cpu"):
    model.to(device)
    model.eval()
    preds_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x in loader:
            batch_x = batch_x.to(device)
            
            output = model.forward(batch_x).to("cpu")
            preds_all = torch.cat((preds_all, output), dim=0)
    return preds_all

In [ ]:
# Checking if GPU is available
if torch.cuda.is_available():
    my_device = "cuda"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

In [ ]:
input_size = train.shape[1]
test_tensor = torch.tensor(test).float()
test_DL = DataLoader(test_tensor, batch_size=2048)

In [ ]:
%%time 
# K-Fold Cross Validation
max_learning_rate = 0.01
epochs = 30
pred_list = []

kf = KFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train)):
    print(f"** fold: {fold+1} ** ........training ...... \n")
    
    # Initialize Model
    model_dnn = DNN(input_size)
    
    # Prepare Data
    X_train, X_valid = train[idx_tr], train[idx_va]
    y_train, y_valid = y[idx_tr], y[idx_va]
    
    ## Converting train and validation labels into tensors
    X_train_tensor = torch.tensor(X_train).float()
    X_valid_tensor = torch.tensor(X_valid).float()
    y_train_tensor = torch.tensor(y_train)
    y_valid_tensor = torch.tensor(y_valid)

    ## Creating train and validation tensors
    train_DS = TensorDataset(X_train_tensor, y_train_tensor)
    valid_DS = TensorDataset(X_valid_tensor, y_valid_tensor)

    ## Defining the dataloaders
    train_DL = DataLoader(train_DS, batch_size=2048, shuffle=True)
    valid_DL = DataLoader(valid_DS, batch_size=2048)
    
    # criterion, optimizer, scheduler
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model_dnn.parameters(), lr=max_learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                              max_lr = max_learning_rate,
                                              epochs = epochs,
                                              steps_per_epoch = len(train_DL),
                                              pct_start = 0.01,
                                              anneal_strategy = "cos")
    
    # Training
    train_losses, valid_losses, learning_rates = train_model(model = model_dnn,
                                                 trainloader = train_DL,
                                                 validloader = valid_DL,
                                                 criterion = criterion,
                                                 optimizer = optimizer,
                                                 scheduler = scheduler,
                                                 epochs = epochs,
                                                 device = my_device,
                                                 print_every = round(epochs/2)-1)
#     break # test
    model_dnn.load_state_dict(torch.load("best-state.pt"))
    pred_test = prediction(model_dnn, test_DL, device=my_device)
    pred_list.append(pred_test.tolist())
#     pred_test_rank = scipy.stats.rankdata(pred_test.tolist())
#     pred_list.append(pred_test_rank)

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(range(epochs), train_losses, label='Train Loss')
ax1.plot(range(epochs), valid_losses, label='Valid Loss')
ax1.set_title('Learning Curve')
ax1.set_xlabel("Number of Epochs")
ax1.set_ylabel("Loss")
ax2 = ax1.twinx()
ax2.set_ylabel("Learning Rate")
ax2.plot(range(epochs), learning_rates, label='Learning Rate', color='g')
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)
plt.show()

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv")
submission['target'] = np.array(pred_list).mean(axis=0)
submission.to_csv('submission.csv', index=False)
submission